In [22]:
# Code for data by individual incident number
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
import csv
from api_key import g_key

csvpath = "Resources/sfpd_incidents_2016.csv"
sfpd_df = pd.read_csv(csvpath)

In [23]:
# The data frame with all unwanted categories removed
sf_clean = sfpd_df[~sfpd_df['Category'].isin(['NON-CRIMINAL', 'OTHER OFFENSES','SECONDARY CODES','WARRANTS','SUICIDE','BRIBERY','EXTORTION','EXTORTION','FAMILY OFFENSES','LOITERING','SEX OFFENSES', 'NON FORCIBLE','BAD CHECKS','GAMBLING','PORNOGRAPHY/OBSCENE MAT','TREA'])]
sf_clean_df = sf_clean.dropna(how='any')
sf_crime = sf_clean_df.drop(['PdId'], axis = 1)

In [24]:
# Cleaning the 'Time' column
time_min = pd.to_datetime(sf_crime["Time"])
time_clean = time_min.dt.hour
sf_crime['Hour'] = time_clean
sf_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",21
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",10
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)",22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150492,160823994,ASSAULT,BATTERY,Sunday,10/09/2016 12:00:00 AM,21:15,MISSION,NONE,18TH ST / MISSION ST,-122.419360,37.761836,"(37.7618358012376, -122.419360352761)",21
150493,160858327,ASSAULT,BATTERY,Friday,10/21/2016 12:00:00 AM,13:30,MISSION,NONE,3400 Block of 18TH ST,-122.419749,37.761867,"(37.7618672648972, -122.419748993977)",13
150495,161061000,ASSAULT,BATTERY,Friday,12/30/2016 12:00:00 AM,21:01,PARK,NONE,OAK ST / STANYAN ST,-122.453982,37.771428,"(37.7714278595913, -122.453981622365)",21
150497,176000758,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,12/29/2016 12:00:00 AM,20:00,CENTRAL,NONE,PINE ST / TAYLOR ST,-122.412269,37.790673,"(37.7906727649886, -122.41226909106)",20


In [25]:
# Creating the bins for the 'Hours'
bins = [0,8,17,24]
time_of_day = ["Morning", "Middle", "Night"]

In [26]:
# Applying the bins to 'Hours'
pd.cut(sf_crime["Hour"], bins, labels=time_of_day)

0         Middle
1         Middle
5          Night
9         Middle
10         Night
           ...  
150492     Night
150493    Middle
150495     Night
150497     Night
150498    Middle
Name: Hour, Length: 104928, dtype: category
Categories (3, object): [Morning < Middle < Night]

In [27]:
# Adding the assigning the bins to 'Time of Day' and adding it to the dataframe
sf_crime["Time of Day"] = pd.cut(sf_crime["Hour"], bins, labels=time_of_day)
sf_crime.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour,Time of Day
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",21,Night
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",10,Middle
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)",22,Night


In [31]:
#Data frame for morning only
morning_crime = sf_crime.loc[sf_crime["Time of Day"]== "Morning"]
morning_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour,Time of Day
13,140776777,ASSAULT,AGGRAVATED ASSAULT WITH A GUN,Thursday,09/15/2016 12:00:00 AM,07:40,INGLESIDE,"ARREST, BOOKED",SILVER AV / CAMELLIA AV,-122.432326,37.729271,"(37.7292705199592, -122.432325871028)",7,Morning
14,160004069,BURGLARY,"BURGLARY,STORE UNDER CONSTRUCTION, FORCIBLE ENTRY",Saturday,01/02/2016 12:00:00 AM,01:43,CENTRAL,NONE,100 Block of SANSOME ST,-122.400909,37.791643,"(37.791642982384, -122.40090869889)",1,Morning
22,160005841,DRUG/NARCOTIC,POSSESSION OF NARCOTICS PARAPHERNALIA,Sunday,01/03/2016 12:00:00 AM,06:50,MISSION,"ARREST, BOOKED",23RD ST / FOLSOM ST,-122.414234,37.754099,"(37.7540986882068, -122.414233849038)",6,Morning
52,160010890,VEHICLE THEFT,STOLEN TRUCK,Tuesday,01/05/2016 12:00:00 AM,05:25,NORTHERN,NONE,HAYES ST / POLK ST,-122.418045,37.777512,"(37.7775118895695, -122.418045452768)",5,Morning
55,160011177,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",Monday,01/04/2016 12:00:00 AM,08:00,TENDERLOIN,NONE,300 Block of EDDY ST,-122.412931,37.783834,"(37.7838344374141, -122.412930522059)",8,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150445,160925368,VEHICLE THEFT,STOLEN TRUCK,Wednesday,11/09/2016 12:00:00 AM,04:45,BAYVIEW,NONE,OAKDALE AV / LANE ST,-122.388483,37.733536,"(37.7335359662276, -122.388483091972)",4,Morning
150455,160980306,ROBBERY,"ROBBERY, BODILY FORCE",Thursday,12/01/2016 12:00:00 AM,08:00,INGLESIDE,NONE,0 Block of PERSIA AV,-122.436264,37.723418,"(37.7234182483189, -122.436263512247)",8,Morning
150458,161017188,ASSAULT,BATTERY,Wednesday,12/14/2016 12:00:00 AM,08:30,BAYVIEW,NONE,1800 Block of EGBERT AV,-122.400756,37.727208,"(37.727208031298, -122.400756491094)",8,Morning
150466,166080328,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Tuesday,04/05/2016 12:00:00 AM,01:19,TARAVAL,NONE,100 Block of GELLERT DR,-122.492860,37.730025,"(37.7300253397345, -122.492860419353)",1,Morning


In [35]:
# Heat map of Morning crime
gmaps.configure(api_key=g_key)

locations = morning_crime[["Y", "X"]].astype(float)
IncidntNum = morning_crime['IncidntNum']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=IncidntNum, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Heat map of crime committed in the middle of the day

In [36]:
middle_crime = sf_crime.loc[sf_crime["Time of Day"]== "Middle"]
middle_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour,Time of Day
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",10,Middle
15,160004150,STOLEN PROPERTY,"STOLEN CHECKS, POSSESSION",Saturday,01/02/2016 12:00:00 AM,11:54,SOUTHERN,"ARREST, BOOKED",900 Block of MARKET ST,-122.408595,37.783707,"(37.7837069301545, -122.408595110869)",11,Middle
16,160004241,ROBBERY,"ROBBERY, ARMED WITH A KNIFE",Saturday,01/02/2016 12:00:00 AM,14:11,MISSION,NONE,900 Block of POTRERO AV,-122.406870,37.757290,"(37.7572895904578, -122.406870402082)",14,Middle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150479,166167891,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,07/22/2016 12:00:00 AM,09:30,PARK,NONE,14TH ST / NOE ST,-122.433409,37.767460,"(37.7674598906238, -122.433409294261)",9,Middle
150486,166285910,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Tuesday,11/22/2016 12:00:00 AM,13:58,CENTRAL,NONE,200 Block of STOCKTON ST,-122.406659,37.788275,"(37.7882745285785, -122.406658711008)",13,Middle
150490,160661158,ASSAULT,BATTERY,Tuesday,08/16/2016 12:00:00 AM,10:00,BAYVIEW,NONE,1300 Block of EVANS AV,-122.382742,37.739945,"(37.7399451834026, -122.382742108888)",10,Middle
150493,160858327,ASSAULT,BATTERY,Friday,10/21/2016 12:00:00 AM,13:30,MISSION,NONE,3400 Block of 18TH ST,-122.419749,37.761867,"(37.7618672648972, -122.419748993977)",13,Middle


In [37]:
# Heat map of Morning crime
gmaps.configure(api_key=g_key)

locations = middle_crime[["Y", "X"]].astype(float)
IncidntNum = middle_crime['IncidntNum']
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=IncidntNum, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
# Heat map of Night crime
night_crime = sf_crime.loc[sf_crime["Time of Day"]== "Night"]
night_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour,Time of Day
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",21,Night
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)",22,Night
20,160005421,FRAUD,FRAUDULENT USE OF AUTOMATED TELLER CARD,Saturday,01/02/2016 12:00:00 AM,21:00,NORTHERN,NONE,GOUGH ST / ROSE ST,-122.422316,37.773619,"(37.7736186276456, -122.422315670749)",21,Night
40,160010486,VANDALISM,"MALICIOUS MISCHIEF, BREAKING WINDOWS",Monday,01/04/2016 12:00:00 AM,21:20,TENDERLOIN,NONE,100 Block of HYDE ST,-122.415670,37.782120,"(37.7821198488931, -122.415669661443)",21,Night
41,160010492,ASSAULT,THREATS AGAINST LIFE,Sunday,01/03/2016 12:00:00 AM,21:30,SOUTHERN,NONE,200 Block of 6TH ST,-122.406346,37.779167,"(37.7791674218963, -122.406346425632)",21,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150470,166108011,FRAUD,"FORGERY, CREDIT CARD",Saturday,05/14/2016 12:00:00 AM,21:22,NORTHERN,NONE,FILLMORE ST / LOMBARD ST,-122.436049,37.799841,"(37.7998412228823, -122.436049203586)",21,Night
150491,160801508,ASSAULT,BATTERY,Sunday,10/02/2016 12:00:00 AM,18:25,SOUTHERN,NONE,200 Block of 8TH ST,-122.411272,37.776033,"(37.7760329501244, -122.411271940537)",18,Night
150492,160823994,ASSAULT,BATTERY,Sunday,10/09/2016 12:00:00 AM,21:15,MISSION,NONE,18TH ST / MISSION ST,-122.419360,37.761836,"(37.7618358012376, -122.419360352761)",21,Night
150495,161061000,ASSAULT,BATTERY,Friday,12/30/2016 12:00:00 AM,21:01,PARK,NONE,OAK ST / STANYAN ST,-122.453982,37.771428,"(37.7714278595913, -122.453981622365)",21,Night


In [40]:
# Heat map of Morning crime
gmaps.configure(api_key=g_key)

locations = night_crime[["Y", "X"]].astype(float)
IncidntNum = night_crime['IncidntNum']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=IncidntNum, 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))